# Predicting if Someone has Tried Cocaine
## Model Building

In [1]:
import pandas as pd
import numpy as np
import pickle

# Data Preprocessing

Before we can use our data to train models, we need to do a few things:

1. Select our desired features

2. Use SimpleImputer to replace NaN values with the mean of that column

3. Apply standard scaling to our numerical variables

4. Dummify/One Hot Encode our categorical variables

We begin with removing year, irwrkstat, cocever, and crkever columns. These columns were only used for EDA, feature engineering, or are redundant.

In [2]:
# Read in pickled data, and drop unneeded columns
df = pd.read_pickle("./pickle/NSDUH_target_dropna_2016-2019.pkl")
df = df.drop(['cocever', 'crkever', 'year', 'irwrkstat'], axis=1)
df

,cig30use,iralcfy,irmjfy,irherfy,irmethamyfq,health,irsex,ireduhighst2,catag3,wrkdhrswk2,irhhsiz2,irki17_2,irpinc3,coccrkever
0,0.0,5,0,0,0,2.0,1,7,1,0.0,1,2,1,0.0
1,0.0,52,364,0,0,1.0,1,8,4,40.0,4,3,2,1.0
2,0.0,48,0,0,0,2.0,0,11,3,0.0,1,1,1,0.0
3,0.0,2,0,0,0,3.0,0,4,1,NaN,5,4,1,0.0
4,22.0,6,0,0,0,3.0,0,9,2,0.0,4,3,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282763,0.0,104,2,0,0,2.0,0,9,2,40.0,2,1,3,0.0
282764,0.0,10,0,0,0,2.0,0,11,3,26.0,2,1,2,0.0
282765,0.0,0,0,0,0,2.0,1,4,1,NaN,6,3,1,0.0
282766,0.0,1,0,0,0,2.0,0,11,1,0.0,6,4,1,0.0


Now, we must separate our numerical and categorical variables. Numerical variables will be adjusted per column by StandardScaler(), which converts the data such that the mean and standard deviation is 0 and 1 for that column, respectively. This standardization across numerical variables increases our model's accuracy.

As for categorical variables, each unique value in a categorical column must be given its own separate, binary column indicating if that observation fits that unique value or not. We do this because keeping them in one column implies some kind of order. Something like gender makes no sense to order, and is therefore a candidate to be separated into different columns (dummified). 

In [3]:
# Continuous variables
num_cols = [
    "iralcfy",
    "catag3",
    "health",
    "ireduhighst2",
    "irpinc3",
    "irki17_2",
    "irmjfy",
    "wrkdhrswk2",
    'irhhsiz2',
    'cig30use',
    'irherfy',
    'irmethamyfq'
]

num_indexes = [i for i in range(12)]

# Categorical variables
cat_cols = ['irsex']

cat_indexes = [12]

# Creating our Column Transformer for our Pipeline

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [6]:
# Create a preprocessor from ColumnTransformer.
# StandardScaler() applied to num_cols, and OneHotEncoder() applied to cat_cols
# We specify cols with indices because column names will be removed when as apply
# our transformer to a pipeline.
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_indexes),
    ('cat', OneHotEncoder(drop='first'), cat_indexes)
])

# Model Building

Now that our data is properly processed, we can test several models across different hyperparameters using GridSearchCV. We will be testing the following models:

1. RandomForestClassifier()

2. LogisticRegression()

3. svm.LinearSVC()

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.model_selection import train_test_split

In [8]:
# Define feature and target columns
features = num_cols+cat_cols
target = "coccrkever"

# Standard naming conventions for feature/test datasets
X = df[features]
y = df[target]

In [9]:
from sklearn.model_selection import GridSearchCV

In [10]:
# Parameter grid for GridSearchCV
model_grid = {
    'random_forest': {
        'model':RandomForestClassifier(random_state=15, n_jobs=3, n_estimators=500),
        'params': {
            'estimator__max_depth': [11, 12, 13, 14],
            'estimator__criterion':['gini', 'entropy']
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(random_state=15, n_jobs=3),
        'params': {
            'estimator__C': [0.085, 0.09, 0.092],
            'estimator__solver':['lbfgs', 'liblinear'],
        }
    },
    'svm': {
        'model': svm.LinearSVC(random_state=15, max_iter=100000),
        'params': {
            'estimator__C':[0.52, 0.55, 0.6, 0.65]
        }
    }
}

In [11]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer

Our dataset still has NaN values in it, but we need to be careful about how we impute missing information in our GridSearchCV. If we ran our imputer on the whole dataset, our imputer would be fitted with data that will become test data, which would result in data leakage. So, we should only fit our imputer based on the training data.

K-cross fold validation will be used throughout the GridSearchCV, which can take a pipeline as its estimator. Our pipeline will include a SimpleImputer that default imputes with the mean and replaces np.nan values. By doing it this way, our SimpleImputer() is only trained on training folds, and any test NaN values are imputed based on the training data. We also include a FunctionTransformer that rounds off imputed data, as our data has only used whole numbers up to this point.

In [12]:
# List to hold model scores
scores = []

for model_name, model_params in model_grid.items():
    # Create our pipeline
    pipe = Pipeline(steps=[
        ('impute', SimpleImputer()),
        ('round', FunctionTransformer(np.round)),
        ('preprocessor', preprocessor),
        ('estimator', model_params['model'])
    ])

    model = GridSearchCV(estimator=pipe, param_grid=model_params['params'], cv=4, return_train_score=False, refit=True)
    model.fit(X, y)
    scores.append({
        'model': model_name,
        'best_score:': model.best_score_,
        'best_params': model.best_params_
    })

C:\Users\Brennan\anaconda3\envs\cocaine_usage_ml\lib\site-packages\sklearn\linear_model\_logistic.py:1355: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 3.
  warnings.warn("'n_jobs' > 1 does not have any effect when"
C:\Users\Brennan\anaconda3\envs\cocaine_usage_ml\lib\site-packages\sklearn\linear_model\_logistic.py:1355: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 3.
  warnings.warn("'n_jobs' > 1 does not have any effect when"
C:\Users\Brennan\anaconda3\envs\cocaine_usage_ml\lib\site-packages\sklearn\linear_model\_logistic.py:1355: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 3.
  warnings.warn("'n_jobs' > 1 does not have any effect when"
C:\Users\Brennan\anaconda3\envs\cocaine_usage_ml\lib\site-packages\sklearn\linear_model\_logistic.py:1355: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set

In [13]:
# Show scores as df
scores

[{'model': 'random_forest',
  'best_score:': 0.8974318168958297,
  'best_params': {'estimator__criterion': 'gini', 'estimator__max_depth': 12}},
 {'model': 'logistic_regression',
  'best_score:': 0.8909777626888473,
  'best_params': {'estimator__C': 0.085, 'estimator__solver': 'liblinear'}},
 {'model': 'svm',
  'best_score:': 0.8904366830758784,
  'best_params': {'estimator__C': 0.52}}]

# Choosing a Model

Although our random forest model has the highest accuracy, the accuracies are very similar. Let's train a model using the best hyperparameters of each, then look at the classification report of each model to gain better insight into the performance of each model.

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.25, random_state=12)

pipe = Pipeline(steps=[
        ('impute', SimpleImputer()),
        ('round', FunctionTransformer(np.round)),
        ('preprocessor', preprocessor)
    ])

X_train = pipe.fit_transform(X_train, y_train)
X_test = pipe.transform(X_test)

Before we train any of our models, we need to create a baseline model for our classifier. Our goal is to successfully predict if someone has used cocaine or not. So, our dummy classifier would simply work by constantly classifying an observation as "Yes, has tried cocaine." We can calculate this dummy model accuracy by simply dividing the number of "1" in our y_test by the length of y_test.

This would result in 100% recall (everyone who has used cocaine was predicted correctly) but with horrendous precision (~12% of "Yes" predictions were correct). If we wanted to make a guess about someone having had used cocaine before, we want to be as precise as possible, even if it is at the cost of recall.

In [15]:
print("Dummy Model Score: ", y_test[y_test==1].sum()/y_test.shape[0])

Dummy Model Score:  0.11782096984100039


In [16]:
# Train a model for each model type using our best hyperparameters. This is so we can analyze each
# in a classification report

rf = RandomForestClassifier(random_state=15, n_jobs=3, n_estimators=500, max_depth=12, criterion='gini')
rf.fit(X_train, y_train)

lg = LogisticRegression(random_state=15, solver='liblinear',C=0.085)
lg.fit(X_train, y_train)

lsvc = svm.LinearSVC(random_state=15, max_iter=100000, C=0.52)
lsvc.fit(X_train, y_train)

LinearSVC(C=0.52, max_iter=100000, random_state=15)

In [17]:
rf_predict = rf.predict(X_test)
lg_predict = lg.predict(X_test)
lsvc_predict = lsvc.predict(X_test)

In [18]:
from sklearn.metrics import classification_report, accuracy_score

In [19]:
print("Random Forest Score: %f\nLogistic Regression Score: %f\nLinear SVC Score: %f\n" %(accuracy_score(y_test, rf_predict), accuracy_score(y_test, lg_predict), accuracy_score(y_test, lsvc_predict)))
print("Random Forest:\n", classification_report(y_test, rf_predict))
print("Logistic Regression:\n", classification_report(y_test, lg_predict))
print("Linear SVC:\n", classification_report(y_test, lsvc_predict))

Random Forest Score: 0.898051
Logistic Regression Score: 0.891232
Linear SVC Score: 0.890794

Random Forest:
               precision    recall  f1-score   support

         0.0       0.91      0.98      0.94     62363
         1.0       0.68      0.25      0.37      8329

    accuracy                           0.90     70692
   macro avg       0.80      0.62      0.66     70692
weighted avg       0.88      0.90      0.88     70692

Logistic Regression:
               precision    recall  f1-score   support

         0.0       0.90      0.98      0.94     62363
         1.0       0.62      0.20      0.30      8329

    accuracy                           0.89     70692
   macro avg       0.76      0.59      0.62     70692
weighted avg       0.87      0.89      0.87     70692

Linear SVC:
               precision    recall  f1-score   support

         0.0       0.90      0.99      0.94     62363
         1.0       0.65      0.16      0.26      8329

    accuracy                         

## Accuracy vs Precision vs Recall

Although the random forest performed the best in terms of total accuracy, our linear SVC model has the highest precision of each model. Recall teh differences between accuracy, precision, and recall:

1. **Accuracy**: Proportion of correct predictions from total observations

2. **Precision**: For a given class, the proportion of correct predictions from total predictions

3. **Recall**: For a given class, proportion of correct predictions from the total number of true observations for that class

Our models have low recall. That means we miss a large number of people who have actually used cocaine. However, we also have extremely high precision. This means that for the people we *do* predict have used cocaine, we are actually correct! This is important to consider. If your goal is to either help people using cocaine or prevent people from becoming addicted cocaine, it would be very bad to wrongly approach someone believing they've tried cocaine when they actually have not. **To prevent false positives, we will choose our Random Forest Classifier because it has the highest precision.**

# Saving our Data
Although we've decided on using the linear SVC model, we will save all the models regardless, just in case we want them in the future.

In [20]:
# Pickle models for later
for model, name in zip([lg, lsvc], ["logreg_model", "lsvc_model"]):
    with open("model/" + name + ".pickle", 'wb') as f:
        pickle.dump(model, f)

In [21]:
import gzip, pickletools

# The output of a regular pickle.dump for our random forest is quite large,
# we can compress it using gzip
with gzip.open("model/randforest_model.pickle", "wb") as f:
    pickled = pickle.dumps(rf)
    optimized_pickle = pickletools.optimize(pickled)
    f.write(optimized_pickle)

"""Code for loading from a gzipped pickle file"""
# with gzip.open("model/randforest_model_optimized.pickle", 'rb') as f:
#     p = pickle.Unpickler(f)
#     rf = p.load()

'Code for loading from a gzipped pickle file'

Next, we need to save our fitted pipeline to transform future data.

In [22]:
# Pickle our Random Forest Classifier
with open("model/pipeline.pickle", 'wb') as f:
        pickle.dump(pipe, f)

Finally, let's save our columns as a JSON file for future reference.

In [23]:
import json

# Firstly we append our data columns to the json
column_info = {
    'data_columns' : [col for col in num_cols+cat_cols]
}

# col_desc is a tab separated text file created separately that contains information for each column
# We'll be sending back this json for users to read with our server, so this information will be helpful
col_desc = pd.read_csv('model/col_desc.txt', header=0, sep='\t')
for row in range(col_desc.shape[0]):
    column_info[col_desc.iloc[row, 0]] = col_desc.iloc[row, 1]

# Write dict to a json file
with open("model/data_columns.json", "w") as f:
    f.write(json.dumps(column_info))

Now, we can move on to creating a server where we can make our model easily interactable.